In [1]:
import os as os 
import skimage.io as io

dataPath = os.path.join('D:\\', 'CMPT_489_Bulk', 'Data' + os.sep)
print(dataPath)
print("Path exists:", os.path.isdir(dataPath))

D:\CMPT_489_Bulk\Data\
Path exists: True


In [2]:
"""
The following example code loops through all the images in the specified dataset
split, creates a binary ground truth mask of each image, and then appends the 
binary ground truth mask to a list. The mask list can then be used for model 
training, analysis, and other tasks.

Participants can set 'write_binary_masks' to True to write the binary mask as 
an 8-bit PNG per the challenge's submission requirements. Ensure to update the 
path as needed. This code assumes the JSON files and image directories are 
stored at the root level of the project directory. 

Final submissions for binary ground truth masks must be of dimensions 720 x 720.
"""
import cv2
import json
import numpy as np

from tqdm.notebook import tqdm

In [9]:
# Change split to "val" or "train" to work on different split. 
split = "train"
images_path = os.path.join(dataPath, "sub_matching_data", f"{split}" + os.sep)
json_path = os.path.join(dataPath, f"VizWiz_SOD_{split}_challenge.json")

print(images_path)
# Change to True to write binary masks as 8-bit PNG files
write_binary_masks = False
write_match_image = False
write_bound_mask = True

dataset = json.load(
    file := open(json_path, "r")
); file.close()

binary_mask_list = []
bound_mask_list = []
for image in tqdm(dataset):
    
    #check if the image is 
    if not (os.path.isfile(images_path + image[:-3] + "png")):
        continue
    original_image = cv2.imread(images_path + image[:-3] + "png")
    json_resized_dimensions = dataset[image]["Ground Truth Dimensions"]

    # Get dimensions of original image and resized image from JSON data
    original_dimensions = original_image.shape
    original_height = int(original_dimensions[0])
    original_width = int(original_dimensions[1])
    resized_height = json_resized_dimensions[0]
    resized_width = json_resized_dimensions[1]

    # Resize image to match JSON data
    resized_image = cv2.resize(
        original_image,
        (resized_width, resized_height),
        interpolation = cv2.INTER_AREA
    )

    # Assert resized image dimensions matches JSON data and create stencil
    resized_dimensions = [resized_image.shape[0], resized_image.shape[1]]
    assert resized_dimensions == json_resized_dimensions
    stencil = np.zeros(resized_dimensions, dtype = np.uint8)
    full_screen = dataset[image]["Full Screen"]

    # If the salient object is full screen then invert the stencil
    if full_screen:
        mask = cv2.bitwise_not(stencil)
        bound = mask.copy()
        bound[1:bound.shape[0]-1, 1:bound.shape[1]-1] = cv2.bitwise_not(bound[1:bound.shape[0]-1, 1:bound.shape[1]-1])
        
        
    
    # Otherwise create a binary mask of the salient object
    else:
        color = [255, 255, 255]
        salient_object_points = dataset[image]["Salient Object"]
        numpy_list = [np.array(polygon) for polygon in salient_object_points]
        copy = stencil.copy()
        mask = cv2.fillPoly(stencil, numpy_list, color)
        bound = cv2.polylines(copy, numpy_list, True, color, 1)
    
    # Append binary mask to binary mask list
    binary_mask_list.append(mask)
    bound_mask_list.append(bound)
    
    
    # Write PNG files if write_binary_masks = True
    if write_bound_mask:
        cv2.imwrite(os.path.join(dataPath, f"sub_boundaries\\{split}\\") + image[0:-3] + "png", bound)

    if write_binary_masks:
        cv2.imwrite(os.path.join(dataPath, f"sub_boundries\\{split}\\") + image[0:-3] + "png", mask)
    
    if write_match_image:
        cv2.imwrite(os.path.join(dataPath, f"matching_data\\{split}\\") + image[0:-3] + "png", resized_image)

D:\CMPT_489_Bulk\Data\sub_matching_data\train\


  0%|          | 0/19116 [00:00<?, ?it/s]